In [2]:
################################################################################
# Program : Train_Taxi_DQN.py
# Description : Open AI GYM의 Taxi 환경의 강화학습 구현 코드 (코랩용) : DQN Taxi
################################################################################

In [3]:
##################################################
# install package
##################################################
!apt-get install -y xvfb x11-utils
!pip install pyvirtualdisplay==0.2.* PyOpenGL==3.1.* PyOpenGL-accelerate==3.1.*

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libfontenc1 libxfont2 libxkbfile1 libxtst6 libxxf86dga1 x11-xkb-utils xfonts-base
  xfonts-encodings xfonts-utils xserver-common
Suggested packages:
  mesa-utils
The following NEW packages will be installed:
  libfontenc1 libxfont2 libxkbfile1 libxtst6 libxxf86dga1 x11-utils x11-xkb-utils xfonts-base
  xfonts-encodings xfonts-utils xserver-common xvfb
0 upgraded, 12 newly installed, 0 to remove and 23 not upgraded.
Need to get 8,045 kB of archives.
After this operation, 12.8 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/main amd64 libfontenc1 amd64 1:1.1.4-1build3 [14.7 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/main amd64 libxfont2 amd64 1:2.0.5-1build1 [94.5 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/main amd64 libxkbfile1 amd64 1:1.1.0-1build3 [71.8 kB]
Get:4 http://archive.ubuntu.

In [4]:
##################################################
# import package
##################################################
import gym
import numpy as np
import random

from keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, Reshape
from keras.optimizers import Adam
from collections import deque
from joblib import dump, load

/usr/local/lib/python3.10/dist-packages/tensorflow/python/framework/dtypes.py:35: DeprecationWarning: ml_dtypes.float8_e4m3b11 is deprecated. Use ml_dtypes.float8_e4m3b11fnuz
  from tensorflow.tsl.python.lib.core import pywrap_ml_dtypes


In [5]:
##################################################
# Define Class and Function
##################################################
# 엡실론그리디 함수
class EpsilonGreedy:
    def __init__(self, max_epsilon, min_epsilon, decay_rate):
        self.max_epsilon = max_epsilon
        self.min_epsilon = min_epsilon
        self.decay_rate = decay_rate
        self.epsilon = max_epsilon

    def exploration_rate(self, step): # 기준값
        self.epsilon = self.min_epsilon + (self.max_epsilon - self.min_epsilon) * np.exp(-self.decay_rate * step)
        return self.epsilon

# DNN 모델 생성
def build_model(input_shape, num_actions):
    model = Sequential()
    model.add(Embedding(input_shape, 10, input_length=1))
    model.add(Reshape((10,)))
    model.add(Dense(50, activation='relu'))
    model.add(Dense(50, activation='relu'))
    model.add(Dense(num_actions, activation='linear'))
    model.compile(loss='mse', optimizer=Adam(lr=0.0001))
    return model


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [6]:
##################################################
# Hyperparameters
##################################################
# 엡실론그리디
MAX_EPSILON = 1.0
MIN_EPSILON = 0.01
DECAY_RATE = 0.001

# 일반 네트워크의 가중치 -> 타겟 네트워크 복제 주기 정의
update_period = 10 # 에피소드 기준
gamma = 0.99  # Discount factor
batch_size = 32
memory = deque(maxlen=100000000) # 최대 저장 공간 for Replay Buffer

In [7]:
##################################################
# Set Envrionment
##################################################
# 환경 정의
# env = gym.make('Taxi-v3').env # time limit 제거 버전
env = gym.make('Taxi-v3')

# state , action 개수 정의
num_actions = env.action_space.n
input_shape = env.observation_space.n
print('state 개수 : {}'.format(input_shape))
print('action 개수 : {}'.format(num_actions))

# 모델 생성
dqn = build_model(input_shape, num_actions)
target_dqn = build_model(input_shape, num_actions)
target_dqn.set_weights(dqn.get_weights())
dqn.summary()

#엡실론그리디
exploration_strategy = EpsilonGreedy(MAX_EPSILON, MIN_EPSILON, DECAY_RATE)


/usr/local/lib/python3.10/dist-packages/gym/core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.10/dist-packages/gym/wrappers/step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(


state 개수 : 500
action 개수 : 6


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 1, 10)             5000      
                                                                 
 reshape (Reshape)           (None, 10)                0         
                                                                 
 dense (Dense)               (None, 50)                550       
                                                                 
 dense_1 (Dense)             (None, 50)                2550      
                                                                 
 dense_2 (Dense)             (None, 6)                 306       
                                                                 
Total params: 8406 (32.84 KB)
Trainable params: 8406 (32.84 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [8]:
##################################################
# Train
##################################################
train_episodes = []
episode_rewards = [] # train_total_rewards
train_total_steps = []
train_epsilon = []

num_episodes = 1000

for episode in range(num_episodes):

    state = env.reset()
    state = np.reshape(state, [1, 1])

    episode_reward = 0
    total_step = 0
    done = False

    while not done: # 종료여부 = False
        # action = epsilon_greedy_action(state)

        # 엡실론그리디
        # rate = exploration_strategy.exploration_rate(total_step)
        rate = exploration_strategy.exploration_rate(episode)

        # 기준치보다 낮으면 랜덤 행동 (엡실론그리디)
        if np.random.rand() < rate:
            action = env.action_space.sample()
        else:
            action = np.argmax(dqn.predict(state, verbose=0))

        # 다음 상태, 보상, 종료여부 정의
        next_state, reward, done, _ = env.step(action)
        next_state = np.reshape(next_state, [1, 1])

        # 경험 저장 (Replay Buffer)
        memory.append((state, action, reward, next_state, done))

        state = next_state
        episode_reward += reward
        # print(f'episode_reward: {episode_reward}, memory: {len(memory)}')
        total_step += 1

        if done:
            break

    # Mini-Batch 학습
    if len(memory) > 3000: # 경험이 3000개 초과 저장되면
    # if len(memory) > batch_size:
        # print(f'train 시작')

        # 32 * 50 = 1600개 Row 학습
        for i in range(50):
            minibatch = random.sample(memory, batch_size)

            for state_m, action_m, reward_m, next_state_m, done_m in minibatch:
                # 일반 네트워크에서의 예측값
                targets = dqn.predict(state_m, verbose=0)

                if not done_m:
                    # s'로 인한 타겟 네트워크에서의 예측값
                    next_q = target_dqn.predict(next_state_m, verbose=0)
                    # 벨먼 방정식 적용
                    targets[0][action] = reward_m + gamma * np.amax(next_q)
                else:
                    targets[0][action] = reward_m
                # 학습
                dqn.fit(state_m, targets, epochs=1, verbose=0)

		# 주기별 일반네트워크의 가중치 타겟 네트워크로 적용
        if episode % update_period == 0 and episode != 0:
            target_dqn.set_weights(dqn.get_weights())

    train_episodes.append(episode)
    episode_rewards.append(episode_reward)
    train_total_steps.append(total_step)
    train_epsilon.append(rate)

    # 주기별로 모델 저장
    if (episode + 1) % 200 == 0:
        target_dqn.save(f'/content/drive/MyDrive/RL/Taxi/taxi_model_e{episode}.h5')

    # 중간에 끊겼을 때 저장용
    dump(episode, '/content/drive/MyDrive/RL/Taxi/taxi_latest_episode.joblib')
    dump(episode_reward, '/content/drive/MyDrive/RL/Taxi/taxi_latest_episode_reward.joblib')
    dump(rate, '/content/drive/MyDrive/RL/Taxi/taxi_latest_exploration_rate.joblib')

    print(f"Episode: {episode + 1}/{num_episodes}, Memory: {len(memory)}, Total Reward: {episode_reward}, Epsilon: {rate}, Total step: {total_step}")

# 마지막 에피소드의 모델 저장
target_dqn.save(f'/content/drive/MyDrive/RL/Taxi/taxi_model_e{episode}.h5')
dump(train_episodes, '/content/drive/MyDrive/RL/Taxi/taxi_train_episodes.joblib')
dump(episode_rewards, '/content/drive/MyDrive/RL/Taxi/taxi_episode_rewards.joblib')
dump(train_total_steps, '/content/drive/MyDrive/RL/Taxi/taxi_train_total_steps.joblib')
dump(train_epsilon, '/content/drive/MyDrive/RL/Taxi/taxi_train_epsilon.joblib')

print("Train Complete!")

Episode: 1/1000, Memory: 200, Total Reward: -731, Epsilon: 1.0, Total step: 200
Episode: 2/1000, Memory: 400, Total Reward: -812, Epsilon: 0.9990104948350412, Total step: 200
Episode: 3/1000, Memory: 600, Total Reward: -740, Epsilon: 0.9980219786806598, Total step: 200
Episode: 4/1000, Memory: 800, Total Reward: -866, Epsilon: 0.9970344505483393, Total step: 200
Episode: 5/1000, Memory: 1000, Total Reward: -731, Epsilon: 0.9960479094505515, Total step: 200
Episode: 6/1000, Memory: 1200, Total Reward: -857, Epsilon: 0.9950623544007555, Total step: 200
Episode: 7/1000, Memory: 1400, Total Reward: -830, Epsilon: 0.9940777844133959, Total step: 200
Episode: 8/1000, Memory: 1600, Total Reward: -893, Epsilon: 0.9930941985039028, Total step: 200
Episode: 9/1000, Memory: 1800, Total Reward: -839, Epsilon: 0.99211159568869, Total step: 200
Episode: 10/1000, Memory: 2000, Total Reward: -776, Epsilon: 0.9911299749851548, Total step: 200
Episode: 11/1000, Memory: 2200, Total Reward: -848, Epsilon:

KeyboardInterrupt: ignored